<a href="https://colab.research.google.com/github/ArjunShenoy14/EasyOCR/blob/main/EasyOCR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install easyocr

In [ ]:
!pip install streamlit

In [168]:
%%writefile app.py
import streamlit as st
import os
import easyocr
import cv2
from matplotlib import pyplot as plt
import numpy as np
import re
import sqlite3
import json
import pandas as pd
import PIL
from PIL import Image

def sqlcon(df2):
  #SQLlite connection
  conn = sqlite3.connect('details.db')
  c = conn.cursor()

  c.execute('''CREATE TABLE IF NOT EXISTS Det(
              Name TEXT,
              Designation TEXT,
              Email TEXT,
              Address TEXT,
              Mobile TEXT
              )''')

  #insert the data into the table
  c.executemany('INSERT INTO Det VALUES(?,?,?,?,?)', zip(df2['Name'], df2['Designation'], df2['Email'], df2['Address'], df2['Mobile']))
  #df.to_sql("Det",conn,if_exists='replace',index=False)
  conn.commit()
  return "Successfully uploaded to database"

def main():

  st.markdown("<h1 style='text-align: center; color: yellow;'>Extracting text using EasyOCR</h1>", unsafe_allow_html=True)


  # Upload image
  uploaded_file = st.file_uploader("Choose an image...", type=["jpg", "jpeg", "png"])

  image_path=""
  if uploaded_file is not None:
      # Display image
      image = Image.open(uploaded_file)
      st.image(image, caption='Uploaded Image', use_column_width=True)
      

      # save the image to disk and retrieve its path
      with open("uploaded_image.jpg", "wb") as f:
          f.write(uploaded_file.getbuffer())
          st.success("Image saved successfully!")
          image_path = f.name
          st.write("Image saved at:", image_path)

      #IMAGE_PATH='/content/images/1.png'
      reader = easyocr.Reader(['en'])

      # Define regular expressions to extract relevant information
      company_name_regex = r'(?:[A-Z][a-z]*\s?)+\b(?:Co\.?|Ltd\.?|Inc\.?|LLC\.?)?\b'
      name_regex = r'(?:[A-Z][a-z]*\s?)+'
      designation_regex = r'(?:[A-Z][a-z]*\s?)+'
      #mobile_regex = r'^\d{3}-\d{3}-\d{4}$'
      mobile_regex = r"\+\d{2}-\d{3}-\d{4}|\d{10}|\+\d{3}-\d{3}-\d{4}|\d{3}-\d{3}-\d{4}"
      email_regex = r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b'
      #url_regex = r'(?:http[s]?://)?(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
      url_regex = r"^https?:\/\/(?:www\.)?[-a-zA-Z0-9@:%._\+~#=]{1,256}\.[a-zA-Z0-9()]{1,6}\b(?:[-a-zA-Z0-9()@:%_\+.~#?&\/=]*)$"
      address_regex = r'(?:[A-Z][a-z]*\s?)+'
      city_regex = r'(?:[A-Z][a-z]*\s?)+'
      state_regex = r'(?:[A-Z][a-z]*\s?)+'
      pin_regex = r'\d{6}'

      # Initialize variables
      #company_name = ''
      name1 = ''
      designation1 = ''
      mobile1 = []
      email1 = ''
      #url = ''
      address1 = ''
      #city = ''
      #state = ''
      #pin = ''

      reader = easyocr.Reader(['en'])
      results = reader.readtext(image_path)
      data=[]
      for i in results:
        data.append(i[1])
      name1=data[0]
      job1=data[1]

      # Loop through the results and extract relevant information
      for result in data:
          text = result
          #if re.match(company_name_regex, text):
          #    company_name = text
          #elif re.match(name_regex, text):
              #name = text
          #elif re.match(designation_regex, text):
              #designation = text
          if re.match(mobile_regex, text):
              mobile1.append(text)
          elif re.match(email_regex, text):
              email1 = text
          elif re.match(address_regex, text):
              address1 = text


      data.remove(name1)
      data.remove(job1)
      data.remove(email1)
      data.remove(address1)
      for i in mobile1:
        data.remove(i)

      mobno = ','.join([str(elem) for elem in mobile1])
      det={'Name':name1, 'Designation':job1,'Email':email1,'Address':address1, 'Mobile':mobno}
      df2=pd.DataFrame([det])
      st.dataframe(df2)
      if st.button('Upload to database'):
        res=sqlcon(df2)
        st.success(res)

if __name__ == '__main__':
	main()

Overwriting app.py


In [ ]:
!pip install pyngrok==4.1.1

In [ ]:
!ngrok authtoken 2KmLhi9o9rhTfgnwUY03qPNSw2p_3nUjkV6j5CBE73Xz2rWy3

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [170]:
!streamlit run --server.port 80 app.py &>/dev/null&
from pyngrok import ngrok 
public_url = ngrok.connect(port='80')
public_url

'http://efdf-35-187-240-2.ngrok.io'

In [169]:
!pgrep streamlit
!ps -eaf | grep streamlit
ngrok.kill()

4470
root        4470       1  2 06:26 ?        00:04:08 /usr/bin/python3 /usr/local/bin/streamlit run --server.port 80 app.py
root       46306     935  0 09:13 ?        00:00:00 /bin/bash -c ps -eaf | grep streamlit
root       46308   46306  0 09:13 ?        00:00:00 grep streamlit
